<a href="https://colab.research.google.com/github/ApoorvBrooklyn/DTL/blob/main/DTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np

n_samples = 1000

np.random.seed(42)

data = {
    'Battery_Capacity_mAh': np.random.randint(1000, 5000, n_samples),
    'Current_Charge_Level_Percent': np.random.randint(0, 100, n_samples),
    'Battery_Age_Years': np.round(np.random.uniform(0, 5, n_samples), 2),
    'Temperature_Celsius': np.random.randint(20, 40, n_samples),
    'Charge_Rate_Amps': np.round(np.random.uniform(0.5, 2.5, n_samples), 2),
    'Charging_Method': np.random.choice(['Fast Charging', 'Normal Charging'], n_samples),
    'Health_Status': np.random.choice(['Good', 'Moderate', 'Poor'], n_samples),
    'Usage_Pattern': np.random.choice(['Heavy Use', 'Light Use', 'Idle'], n_samples)
}

In [32]:
data['Optimal_Charging_Time_Minutes'] = (
    (data['Battery_Capacity_mAh'] - (data['Current_Charge_Level_Percent'] / 100) * data['Battery_Capacity_mAh']) /
    data['Charge_Rate_Amps'] *
    (1 + 0.1 * data['Battery_Age_Years']) *  # Aging factor
    np.where(data['Charging_Method'] == 'Fast Charging', 0.75, 1.0) *  # Fast charging is faster
    np.where(data['Health_Status'] == 'Poor', 1.5, np.where(data['Health_Status'] == 'Moderate', 1.2, 1.0)) *  # Health status impact
    np.where(data['Usage_Pattern'] == 'Heavy Use', 1.25, np.where(data['Usage_Pattern'] == 'Light Use', 1.1, 1.0))  # Usage pattern impact
)

In [34]:
battery_df = pd.DataFrame(data)


battery_df.to_csv('synthetic_battery_dataset.csv', index=False)

In [37]:
import pandas as pd

# Load the dataset
battery_df = pd.read_csv('synthetic_battery_dataset.csv')

# Display the first few rows of the dataset
(battery_df.head())


,Battery_Capacity_mAh,Current_Charge_Level_Percent,Battery_Age_Years,Temperature_Celsius,Charge_Rate_Amps,Charging_Method,Health_Status,Usage_Pattern,Optimal_Charging_Time_Minutes
0,4174,4,3.54,35,2.20,Fast Charging,Poor,Heavy Use,3468.024818
1,4507,32,4.19,20,1.19,Normal Charging,Moderate,Heavy Use,5481.799714
2,1860,64,3.49,22,0.56,Normal Charging,Poor,Light Use,2661.480643
3,2294,17,3.40,39,0.60,Normal Charging,Poor,Idle,6378.467000
4,2130,95,3.09,21,0.78,Fast Charging,Moderate,Light Use,176.941558


GRADIENT BOOST


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Separate features and target variable
X = battery_df.drop('Optimal_Charging_Time_Minutes', axis=1)
y = battery_df['Optimal_Charging_Time_Minutes']

# Encode categorical variables
categorical_features = ['Charging_Method', 'Health_Status', 'Usage_Pattern']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'  # Leave the rest of the columns unchanged
)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

# Create a pipeline with preprocessing and Gradient Boosting Regressor
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=3, random_state=42))
])

# Train the model
gb_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_pipeline.predict(X_test)

# Evaluate the model
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f'Gradient Boosting Regression - Mean Absolute Error: {mae_gb}')
print(f'Gradient Boosting Regression - R-squared: {r2_gb}')


Gradient Boosting Regression - Mean Absolute Error: 234.38480827884086
Gradient Boosting Regression - R-squared: 0.9562464354486508
